## Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Download libraries

%pip install numpy==1.22.4
%pip install tqdm==4.65.0
%pip install nltk==3.8.1
%pip install jsonlines==3.1.0
%pip install transformers==4.26.1
%pip install scikit-learn==1.2.2
%pip install huggingface-hub==0.12.1
%pip install scipy==1.10.1
%pip install urllib3==1.26.15
%pip install six==1.16.0
%pip install torch==1.10.1
%pip install sklearn
%pip install pandas
%pip install datasets
%pip install sentencepiece
%pip install peft
%pip install accelerate
%pip install bitsandbytes
%pip install bnb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in

Run this cell to load the autoreload extension. This allows us to edit .py source files, and re-import them into the notebook for a seamless editing and debugging experience.

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import torch
import torch.utils.data
import random
import numpy as np

# Configure GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Set path
import os
ROOT_PATH = "/content/drive/MyDrive/milestone3/"
# ROOT_PATH = "./"
# Set random seed
seed = 345413
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

import sys
sys.path.append(ROOT_PATH)
     

cuda:0


to use helper functions

In [5]:
sys.path.append(ROOT_PATH)


In [6]:
import json
import jsonlines
from tqdm import tqdm
import random
from copy import deepcopy
import sklearn
import pandas as pd
import sys
import time

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

from helpers import EPFLCourseAssistantDataset
     


## Preprocess Data


In [7]:
data_file = ROOT_PATH + "data/solutions_v1.json"
raw_data = []
answers = []
# start actual function
with open(data_file, "r") as f:
    raw_data = json.load(f)


for row in tqdm(raw_data):
    if "choices" in row.keys() and row["choices"] is not None:
      
        if isinstance(row["answer"], str):
            correct_answers = row["answer"]
        elif isinstance(row["answer"], int):
            if row["answer"] >= len(row["choices"]):
                correct_answers = str(row["answer"])
            else:
                correct_answers = row["choices"][row["answer"]]

        answers.append({
            "answer": str(" ".join(correct_answers) if isinstance(correct_answers, list) else correct_answers),
            "question": str(row["question"]),
            "sol_id": str(row["sol_id"]),
            "is_correct": "true"
        })
        for choice in row["choices"]:
            sanitized_choice = choice if isinstance(choice, str) else row["choices"][choice]
            if sanitized_choice not in correct_answers:
                answers.append({
                    "answer": str(choice if isinstance(choice, str) else row["choices"][choice]),
                    "question": str(row["question"]),
                    "sol_id": str(row["sol_id"]),
                    "is_correct": "false"
                })
    else:
        if "answer" in row.keys():
            answers.append({
                "answer": str(row["answer"]),
                "question": str(row["question"]),
                "sol_id": str(row["sol_id"]),
                "is_correct": "true"
            })


with open(ROOT_PATH + "data/fine_tune_data.json", "w") as f:
    json.dump(answers, f, indent=4)

100%|██████████| 4450/4450 [00:00<00:00, 147368.42it/s]


## Model finetuning

In [8]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import TrainingArguments, Trainer

checkpoint = "google/flan-t5-base"
prefix = "question: "
max_input_length = 512
max_target_length = 512

dataset = load_dataset("json", data_files=ROOT_PATH + "data/fine_tune_data.json")

dataset = dataset.filter(lambda example: example['is_correct'] == 'true')

tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def preprocess_examples(examples):
    answers = examples["answer"]
    questions = examples["question"]
    inputs = [prefix + input for input in questions]
    model_inputs = tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(answers, max_length=max_target_length, padding="max_length", truncation=True).input_ids

    # important we need to replace -100 in the labels as the -100 token id is not part of the labels
    # and thus not taken by the loss function
    labels_with_ignore_index = []
    for labels_example in labels:
        labels_example = [label if label != 0 else -100 for label in labels_example]
        labels_with_ignore_index.append(labels_example)
    
    model_inputs["labels"] = labels_with_ignore_index

    return model_inputs


tokenized_datasets = dataset.map(preprocess_examples, batched=True)

tokenized_datasets.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-8ace029b19627952/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Filter:   0%|          | 0/12591 [00:00<?, ? examples/s]

Map:   0%|          | 0/4087 [00:00<?, ? examples/s]

In [9]:
tokenized_datasets = tokenized_datasets.shuffle(seed=42)['train'].train_test_split(test_size=0.1)
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['is_correct', 'question', 'answer', 'sol_id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3678
    })
    test: Dataset({
        features: ['is_correct', 'question', 'answer', 'sol_id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 409
    })
})

In [13]:
import transformers
from transformers import AutoModelForSeq2SeqLM, Trainer

training_args = TrainingArguments(
    output_dir=ROOT_PATH + 'results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            
    logging_strategy="steps",        
    logging_steps=10,  
)

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, device_map="auto")
print(model.get_memory_footprint())


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/c782cba52f8ea6a704240578055cf1c3fc2f2ca9/config.json
Model config T5Config {
  "_name_or_path": "google/flan-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_lay

990311424


In [14]:
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType

lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM, 
    inference_mode=False, 
    r=8, 
    lora_alpha=32, 
    lora_dropout=0.1
)
# prepare int-8 model for training
# model = prepare_model_for_int8_training(model)

# add LoRA adaptor
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 884736 || all params: 248462592 || trainable%: 0.3560841867092814


In [15]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir=ROOT_PATH + "results",
	  auto_find_batch_size=True,
    learning_rate=1e-3, # higher learning rate
    num_train_epochs=5,

)

trainer = Seq2SeqTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=tokenized_datasets['train'],         # training dataset
    eval_dataset=tokenized_datasets['test']        # evaluation dataset
)

trainer.train()



trainer.save_model(ROOT_PATH + "results")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `PeftModelForSeq2SeqLM.forward` and have been ignored: is_correct, question, answer, sol_id. If is_correct, question, answer, sol_id are not expected by `PeftModelForSeq2SeqLM.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3678
  Num Epochs = 5
  Instantaneous batch s

Step,Training Loss
500,2.891600
1000,2.644500
1500,2.566800
2000,2.502800
2500,2.448300
3000,2.431900
3500,2.420800
4000,2.437200
4500,2.459700


Saving model checkpoint to /content/drive/MyDrive/milestone3/results/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/MyDrive/milestone3/results/checkpoint-1000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/MyDrive/milestone3/results/checkpoint-1500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/MyDrive/milestone3/results/checkpoint-2000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/MyDrive/milestone3/results/checkpoint-2500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/MyDrive/milestone3/results/checkpoint-3000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/MyDrive/milestone3/results/ch

In [16]:
peft_model_id="results"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)

tokenizer config file saved in results/tokenizer_config.json
Special tokens file saved in results/special_tokens_map.json
Copy vocab file to results/spiece.model


('results/tokenizer_config.json',
 'results/special_tokens_map.json',
 'results/spiece.model',
 'results/added_tokens.json',
 'results/tokenizer.json')

## Inference

In [17]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load peft config for pre-trained checkpoint etc.
peft_model_id = "results"
config = PeftConfig.from_pretrained(peft_model_id)

# load base LLM model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path,  load_in_8bit=True,  device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id, device_map={"":0})
model.eval()

print("Peft model loaded")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/c782cba52f8ea6a704240578055cf1c3fc2f2ca9/config.json
Model config T5Config {
  "_name_or_path": "google/flan-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
 

Peft model loaded


In [22]:
from random import randrange


input_ids = tokenizer(prefix + "Why one should interpret FRAP results with caution?", return_tensors="pt", truncation=True).input_ids.cuda()
outputs = model.generate(input_ids=input_ids, top_p=0.9)
print(f"{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]}")

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}



FRAP results are not always accurate
